## Task 1

In [5]:
def find_min_pledge(pledge_list):
    pledges_set = set(pledge_list)
    min_pledge = 1
    while min_pledge in pledges_set:
        min_pledge += 1
    return min_pledge

In [9]:
assert find_min_pledge([1, 3, 6, 4, 1, 2]) == 5
assert find_min_pledge([1, 2, 3]) == 4
assert find_min_pledge([-1, -3]) == 1

## Task 2

In [1]:
import feedparser

google_news_url = "https://news.google.com/news/rss"

def get_headlines(rss_url):
    feed = feedparser.parse(rss_url)
    titles = [entry.title for entry in feed.entries]
    return titles

print(get_headlines(google_news_url))


['Israel strike on Gaza school building kills more than 70, hospital head says - BBC.com', 'EXCLUSIVE: Ex-deputy who killed Sonya Massey had history of complaints involving women - USA TODAY', 'Harris Leads Trump in Three Key States, Times/Siena Polls Find - The New York Times', 'Ukraine updates: IAEA warns of nuclear disaster in Kursk - DW (English)', 'Brazilian authorities are investigating the cause of the fiery plane crash that killed 62 - The Associated Press', "Arizona Kamala Harris rally speakers court 'John McCain Republicans' support; man brings border map for VP - Fox News", 'Trump hits NYT while doubling down on near-crash helicopter story - The Hill', 'Debby finally moves out of the US, though risk from flooded rivers remains - The Associated Press', "California's cracking down hard on unhoused people – and they're running out of options - USA TODAY", 'Tim Walz vs. JD Vance: What the 2024 presidential running mates could mean for your wallet - CNBC', 'U.S. prepares for cruc

## Task 3

In [2]:
def get_payments_storage():
    """
    @returns an instance of https://docs.python.org/3/library/io.html#io.BufferedWriter
    """
    # Sample implementation to make the code run in coderpad.
    # Do not rely on this exact implementation.
    return open('/dev/null', 'wb')

# This is a library function, you can't modify it.
def stream_payments_to_storage(storage):
    """
    Loads payments and writes them to the `storage`.
    Returns when all payments have been written.
    @parameter `storage`: is an instance of
    https://docs.python.org/3/library/io.html#io.BufferedWriter
    """
    # Sample implementation to make the code run in coderpad.
    # Do not rely on this exact implementation.
    for i in range(10):
        storage.write(bytes([1, 2, 3, 4, 5]))


In [3]:
class ChecksumStorage:
    def __init__(self, storage):
        self.storage = storage
        self.checksum = 0

    def write(self, buffer):
        self.checksum += sum(buffer)
        self.storage.write(buffer)

    def get_checksum(self):
        return self.checksum

def process_payments():
    storage = get_payments_storage()
    checksum_storage = ChecksumStorage(storage)
    stream_payments_to_storage(checksum_storage)
    print(checksum_storage.get_checksum())

process_payments()


150


## Task 5

In [4]:
def stream_payments(callback_fn):
    """
    Reads payments from a payment processor and calls `callback_fn(amount)`
    for each payment.
    Returns when there is no more payments.
    """
    # Sample implementation to make the code run in coderpad.
    # Do not rely on this exact implementation.
    for i in range(10):
        callback_fn(i)
        
        
# This is a library function, you can't modify it.
def store_payments(amount_iterator):
    """
    Iterates over the payment amounts from amount_iterator
    and stores them to a remote system.
    """
    # Sample implementation to make the code run in coderpad.
    # Do not rely on this exact implementation.
    for i in amount_iterator:
        print(i)
        
        
def callback_example(amount):
    print(amount)
    return True

In [5]:
def process_payments_2():
    """
    Modify `process_payments_2()`, write glue code that enables
    `store_payments()` to consume payments produced by `stream_payments()`.
    """
    def payment_generator():
        payment_queue = []

        def callback_fn(amount):
            payment_queue.append(amount)

        stream_payments(callback_fn)

        while payment_queue:
            yield payment_queue.pop(0)

    payments = payment_generator()
    store_payments(payments)

process_payments_2()


0
1
2
3
4
5
6
7
8
9


## Task 6

In [21]:
def get_value(data, key, default, lookup=None, mapper=None):
    """
    Finds the value from data associated with key, or default if the
    key isn't present.
    If a lookup enum is provided, this value is then transformed to its
    enum value.
    If a mapper function is provided, this value is then transformed
    by applying mapper to it.
    """
    # Use data.get(key, default) to handle missing keys
    return_value = data[key]  # This will raise a KeyError if key is not in data
    
    # Check if the key exists before using it
    if return_value is None or return_value == "":
        return_value = default

    # Use get() method to handle missing keys
    if lookup:
        return_value = lookup[return_value]  # This may raise a KeyError if the value is not in lookup

    if mapper:
        # Add error handling
        return_value = mapper(return_value)

    return return_value


def ftp_file_prefix(namespace):
    """
    Given a namespace string with dot-separated tokens, returns the
    string with
    the final token replaced by 'ftp'.
    Example: a.b.c => a.b.ftp
    """
    # Handle cases with no dots in the namespace
    if '.' not in namespace:
        raise ValueError("Namespace must contain at least one dot.")

    return ".".join(namespace.split(".")[:-1]) + '.ftp'


def string_to_bool(string):
    """
    Returns True if the given string is 'true' case-insensitive,
    False if it is
    'false' case-insensitive.
    Raises ValueError for any other input.
    """
    if string.lower() == 'true':
        return True
    if string.lower() == 'false':
        return False
    raise ValueError(f'String {string} is neither true nor false')


def config_from_dict(dict):
    """
    Given a dict representing a row from a namespaces csv file,
    returns a DAG configuration as a pair whose first element is the
    DAG name
    and whose second element is a dict describing the DAG's properties
    """
    # Don't use dict as a variable name to prevent conflicts with the built-in Python dictionary type
    namespace = dict['Namespace']
    return (dict['Airflow DAG'],
            {"earliest_available_delta_days": 0,
             "lif_encoding": 'json',
             "earliest_available_time":
                 get_value(dict, 'Available Start Time', '07:00'),
             "latest_available_time":
                 get_value(dict, 'Available End Time', '08:00'),
             "require_schema_match":
                 get_value(dict, 'Requires Schema Match', 'True',
                           mapper=string_to_bool),
             "schedule_interval":
                 get_value(dict, 'Schedule', '1 7 * * * '),
             "delta_days":
                 get_value(dict, 'Delta Days', 'DAY_BEFORE',
                           lookup=DeltaDays),  
             "ftp_file_wildcard":
                 get_value(dict, 'File Naming Pattern', None),
             "ftp_file_prefix":
                 get_value(dict, 'FTP File Prefix',
                           ftp_file_prefix(namespace)),  
             "namespace": namespace
             }
           )
